In [2]:
from ultralytics import YOLO
import cv2, os, glob 
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

from sort.sort import *
from util import *

# link->https://www.youtube.com/watch?v=fyJB1t0o0ms [for ANPR]

Using CPU. Note: This module is much faster with a GPU.


In [3]:
cocomodel=YOLO('yolov8n.pt')
# print(cocomodel)
# linik->https://www.youtube.com/watch?v=m9fH9OWn8YM  [for model training]

In [4]:
license_plate_detector=YOLO("C:\\Users\\Admin\\Desktop\\test1\\ANPR\\test2\\weights\\best.pt")
# print(license_plate_detector)

In [5]:
vehicle=[2,3,4,6,8]         #list of label id for vechil detection 

mater_tracker=Sort()        #used for tracking vechiles

resultes={}

crop_lp=[]
crop_lp_list_gray=[]
crop_lp_list_threshold=[]

In [10]:
# video 
vid=cv2.VideoCapture("C:/Users/Admin/Desktop/test1/ANPR/test2/sample.mp4")

# frame_count=int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
# W=int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
# H=int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
# FPS=int(vid.get(cv2.CAP_PROP_FPS))
# print(f"frame count:{frame_count}\nframe width:{W}\nframe hight:{H}\nFPS:{FPS}")

# videowriter=cv2.VideoWriter("C:/Users/Admin/Desktop/test1/ANPR/test2/out_sample.mp4",cv2.VideoWriter_fourcc('F','M','P','4'),FPS,(W,H),0)

ret,frame_no=True,-1
while(ret):
    frame_no+=1
    ret,frame=vid.read()
    print(ret,frame_no)
    if(ret):
        # if(frame_no>10): break
        resultes[frame_no]={}

        # detect vechile
        detections= cocomodel(frame)[0]
        detections_list=[]
        # print(detections)
        # detectic  vechile
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if(int(class_id) in vehicle):
                detections_list.append([x1,y1,x2,y2,score])
        
        # track vechiles
        track_id=mater_tracker.update(np.asarray(detections_list))
        # print(track_id)
        # detect license plate 
        license_plates=license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            # print(license_plate)
            x1,y1,x2,y2,score,class_id=license_plate
            # print(type(x1),type(y1),type(x2),type(y2),type(score),type(class_id)
            
            # assign licences plate to car
            xcar1,ycar1,xcar2,ycar2,car_id=get_car(license_plate,track_id)
            # corp license plate
            corp_license_plate=frame[int(y1):int(y2),int(x1):int(x2),:]
            # process licens plate
            corp_license_plate_gray=cv2.cvtColor(corp_license_plate,cv2.COLOR_BGR2GRAY)
            _,corp_license_plate_therhold=cv2.threshold(corp_license_plate,64,255,cv2.THRESH_BINARY_INV)

            # cv2.imshow("orignal_crop",corp_license_plate)
            # cv2.imshow("thresh_orignal_crop",corp_license_plate_therhold)
            
            crop_lp.append(corp_license_plate)
            crop_lp_list_gray.append(corp_license_plate_gray)
            crop_lp_list_threshold.append(corp_license_plate_therhold)

            # print(corp_license_plate_therhold)            #debuging

            # read lincese plate
            license_plate_text,license_plate_score=read_license_plate(corp_license_plate_therhold)
            # print(license_plate_text,license_plate_score)     #debugging

            #update frame to show
            # out_frame=cv2.putText(frame,text=license_plate_text,org=(x1,y1),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.5,color=(0,255,255),thickness=1,lineType=cv2.LINE_AA)
            # out_frame=cv2.rectangle(out_frame,(x1,y2),(x1,y2),(0,255,0),2)

            # videowriter.write(out_frame)

            if(license_plate_text is not None):
                resultes[frame_no][car_id]={'car':{'bbox':[xcar1,ycar1,xcar2,ycar2]},
                                            'license_plate':{'bbox':[ x1,y1,x2,y2],'text':license_plate_text,'bbox_score': score,'text_score':license_plate_score}}

# write resulte
write_csv(resultes,"C:\\Users\\Admin\\Desktop\\test1\\ANPR\\test2\\test.csv")
            
vid.release()


0: 384x640 21 cars, 1 bus, 2 trucks, 106.0ms
Speed: 3.0ms preprocess, 106.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



True 0


0: 384x640 2 License_Plates, 113.0ms
Speed: 3.0ms preprocess, 113.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 90.0ms
Speed: 4.0ms preprocess, 90.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)



True 1


0: 384x640 2 License_Plates, 108.0ms
Speed: 3.0ms preprocess, 108.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 111.0ms
Speed: 5.0ms preprocess, 111.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



...

0: 384x640 6 License_Plates, 102.0ms
Speed: 2.4ms preprocess, 102.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



True 819


0: 384x640 22 cars, 2 trucks, 189.6ms
Speed: 8.7ms preprocess, 189.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 License_Plates, 123.9ms
Speed: 3.0ms preprocess, 123.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 2 trucks, 118.2ms
Speed: 11.5ms preprocess, 118.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)



True 820


0: 384x640 4 License_Plates, 107.3ms
Speed: 9.2ms preprocess, 107.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


In [1]:
df=pd.read_csv("C:\\Users\\Admin\\Desktop\\test1\\ANPR\\test2\\test.csv")
df

NameError: name 'pd' is not defined

In [11]:
print(len(crop_lp))
print(crop_lp[1])
plt.imshow(crop_lp[1])

25
[[[57 44 33]
  [55 42 31]
  [55 42 31]
  ...
  [48 36 29]
  [48 36 29]
  [46 34 27]]

 [[62 50 36]
  [62 50 36]
  [62 49 38]
  ...
  [52 40 33]
  [51 39 32]
  [50 38 31]]

 [[67 55 41]
  [67 55 41]
  [67 54 43]
  ...
  [55 43 36]
  [53 41 34]
  [53 41 34]]

 ...

 [[44 35 32]
  [44 35 32]
  [44 35 32]
  ...
  [36 30 25]
  [36 30 25]
  [36 30 25]]

 [[44 35 32]
  [44 35 32]
  [44 35 32]
  ...
  [35 29 24]
  [35 29 24]
  [35 29 24]]

 [[45 36 33]
  [45 36 33]
  [45 36 33]
  ...
  [33 27 22]
  [33 27 22]
  [33 27 22]]]


NameError: name 'cv2_imshow' is not defined